# 時間データ処理

この章では、時間データの処理に焦点を当て、時間ベースのグループ化や集計、時間ウィンドウの操作方法について説明します。

In [2]:
import polars as pl
from helper.jupyter import row

時間データを効率的に扱うために、Polarsでは日付や時刻、時間間隔に関連する4つのデータ型を提供しています。それぞれのデータ型には異なる特徴と用途があり、時間ベースのデータ処理において重要な役割を果たします。このセクションでは、以下の4つの時間データ型について説明します。

| データ型   | 概要                                | 最小単位    | ゼロ点                                       | 用途                                       | 例                           |
|------------|-------------------------------------|-------------|---------------------------------------------|--------------------------------------------|------------------------------|
| **`Date`** | 日付を表すデータ型、時刻情報なし   | 日          | Unix epoch (1970-01-01)                    | 日付のみが重要なデータの管理や処理          | `2024-07-15`                 |
| **`Time`** | 1日の中の時刻を表すデータ型        | ナノ秒(ns)| 00:00:00（1日の開始時刻）                  | 1日の中の時刻や時間間隔を扱う               | `12:30:45.123456789`         |
| **`Datetime`** | 日付と時刻を組み合わせたデータ型 | ミリ秒(ms)・マイクロ秒(us)・ナノ秒(ns) | Unix epoch (1970-01-01 00:00:00)          | 時系列データやタイムスタンプの処理          | `2024-07-15 12:30:45.123456` |
| **`Duration`** | 時間の長さや間隔を表すデータ型    | ミリ秒(ms)・マイクロ秒(us)・ナノ秒(ns) | -                                           | 2つの時点間の差分や時間間隔の計算          | `2 days`, `3 hours`, `45 minutes` |

次の演算式関数で、複数の整数列から時間列に変換することができます。


- **`pl.date()`**  
  `pl.date()`は、年、月、日を指定して`Date`型を作成します。これにより、`year`、`month`、`day`の列を使用して日付データを作成します。

- **`pl.time()`**  
  `pl.time()`は、時、分、秒を指定して`Time`型を作成します。ここでの演算式は、秒数をマイクロ秒に変換し、`microsecond`引数に適切に丸めて渡しています。これにより、`hour`、`minute`、`second`を使って`Time`型のデータを生成します。

  - `microsecond=pl.col('second').mod(1).mul(1e6).round()`: `second`列の小数部を取得し、それをマイクロ秒単位に変換しています。

- **`pl.datetime()`**  
  `pl.datetime()`は、年、月、日、時、分、秒を指定して`Datetime`型を作成します。こちらも、秒の小数部をマイクロ秒に変換し、適切に丸めて`microsecond`に渡しています。

- **`pl.duration()`**  
  `pl.duration()`は、時間の長さを`Duration`型として表現します。この場合、`total_seconds`列を使って、秒単位からマイクロ秒に変換した時間間隔を作成しています。

  - `microseconds=pl.col('total_seconds').mul(1e6)`: `total_seconds`列をマイクロ秒単位に変換しています。

In [3]:
df_numbers = pl.DataFrame(
    dict(
        year=[2022, 2023, 2024],
        month=[11, 10, 12],
        day=[4, 31, 2],
        hour=[1, 6, 16],
        minute=[10, 50, 34],
        second=[12.2, 20.5, 21],
    )
)

df_numbers = df_numbers.with_columns(
    total_seconds=pl.col('hour') * 3600 + pl.col('minute') * 60 + pl.col('second')
)

次のコードでは、`pl.date()`, `pl.time()`, `pl.datetime()`, `pl.duration()`を使用して、整数列から時間列への変換を行い、それぞれの時間データ型に対応したデータを生成しています。

`Date`、`Time`、`Datetime`の場合、各フィールドには値の範囲があり、上限を超えないように注意する必要があります。また、各フィールドが整数でない場合は、整数部分のみを使用します。

❶浮動小数点の秒列からマイクロ秒に換算します。❷`Duration`の場合は、上限がないため、`microseconds`を使用してマイクロ秒単位の時間差を作成します。

In [4]:
df_times = df_numbers.select(
    date=pl.date('year', 'month', 'day'),
    time=pl.time('hour', 'minute', 'second', microsecond=pl.col('second').mod(1).mul(1e6).round()), #❶
    datetime=pl.datetime('year', 'month', 'day', 'hour', 'minute', 'second', 
                         microsecond=pl.col('second').mod(1).mul(1e6).round()),
    duration=pl.duration(microseconds=pl.col('total_seconds').mul(1e6)), #❷
)

row(df_numbers, df_times)

shape: (3, 7) year month day hour minute second total_seconds i64 i64 i64 i64 i64 f64 f64 2022 11 4 1 10 12.2 4212.2 2023 10 31 6 50 20.5 24620.5 2024 12 2 16 34 21.0 59661.0 
 shape: (3, 4) date time datetime duration date time datetime[μs] duration[μs] 2022-11-04 01:10:12.200 2022-11-04 01:10:12.200 1h 10m 12s 200ms 2023-10-31 06:50:20.500 2023-10-31 06:50:20.500 6h 50m 20s 500ms 2024-12-02 16:34:21 2024-12-02 16:34:21 16h 34m 21s

次のコードでは、異なる時間型を変換および操作する例です。以下に各部分の説明を示します。

❶ `dt.combine()`で`date`列と`time`列を結合して、`Datetime`型の列を作成します。<br>
❷ `dt.date()`で`datetime`列から日付部分だけを抽出し、`Date`型の列を作成します。<br>
❸ `dt.time()`で`datetime`列から時刻部分だけを抽出し、`Time`型の列を作成します。<br>
❹ `cast(pl.Duration)`で`time`列を`Duration`型にキャストします。<br>
❺ `Duration`型を直接`Time`型に変換する関数がないため、一旦列`dt.cast_time_unit('ns')`を使用してをナノ秒単位の`Duration`型に変換し、`to_physical()`で`Int64`型に変換後、`cast(pl.Time)`で`Time`型に変換します。

In [5]:
df_times.select(
    datetime=pl.col('date').dt.combine(pl.col('time')), #❶
    date=pl.col('datetime').dt.date(), #❷
    time=pl.col('datetime').dt.time(), #❸
    duration=pl.col('time').cast(pl.Duration), #❹
    time2=pl.col('duration').dt.cast_time_unit('ns').to_physical().cast(pl.Time), #❺
)

datetime,date,time,duration,time2
datetime[μs],date,time,duration[μs],time
2022-11-04 01:10:12.200,2022-11-04,01:10:12.200,1h 10m 12s 200ms,01:10:12.200
2023-10-31 06:50:20.500,2023-10-31,06:50:20.500,6h 50m 20s 500ms,06:50:20.500
2024-12-02 16:34:21,2024-12-02,16:34:21,16h 34m 21s,16:34:21


In [106]:
df_times.select(
    year=pl.col('datetime').dt.year(),
    month=pl.col('datetime').dt.month(),
    day=pl.col('datetime').dt.day(),
    hour=pl.col('datetime').dt.hour(),
    minute=pl.col('datetime').dt.minute(),
    second=pl.col('datetime').dt.second(),
    microsecond=pl.col('datetime').dt.microsecond()
)

year,month,day,hour,minute,second,microsecond
i32,i8,i8,i8,i8,i8,i32
2022,11,4,1,10,12,200000
2023,10,31,6,50,20,500000
2024,12,2,16,34,21,0


In [107]:
df_times.select(
    pl.col('duration'),
    days=pl.col('duration').dt.total_days(),
    hours=pl.col('duration').dt.total_hours(),
    minutes=pl.col('duration').dt.total_minutes(),
    seconds=pl.col('duration').dt.total_seconds(),
    microseconds=pl.col('duration').dt.total_microseconds()
)

duration,days,hours,minutes,seconds,microseconds
duration[μs],i64,i64,i64,i64,i64
1h 10m 12s 200ms,0,1,70,4212,4212200000
6h 50m 20s 500ms,0,6,410,24620,24620500000
16h 34m 21s,0,16,994,59661,59661000000


In [108]:
df_times.select(
    month_start=pl.col('datetime').dt.month_start(),
    month_end=pl.col('datetime').dt.month_end(),
)

month_start,month_end
datetime[μs],datetime[μs]
2022-11-01 01:10:12.200,2022-11-30 01:10:12.200
2023-10-01 06:50:20.500,2023-10-31 06:50:20.500
2024-12-01 16:34:21,2024-12-31 16:34:21


In [109]:
df_times.select(
    week=pl.col('datetime').dt.week(),
    weekday=pl.col('datetime').dt.weekday(),
)

week,weekday
i8,i8
44,5
44,2
49,1


In [110]:
df_times.select(
    'datetime',
    t_1s=pl.col('datetime').dt.truncate('1s'),
    t_30s=pl.col('datetime').dt.truncate('30s'),
    t_1h=pl.col('datetime').dt.truncate('1h'),
    t_1d=pl.col('datetime').dt.truncate('1d'),
    t_1mo=pl.col('datetime').dt.truncate('1mo'),
)

datetime,t_1s,t_30s,t_1h,t_1d,t_1mo
datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2022-11-04 01:10:12.200,2022-11-04 01:10:12,2022-11-04 01:10:00,2022-11-04 01:00:00,2022-11-04 00:00:00,2022-11-01 00:00:00
2023-10-31 06:50:20.500,2023-10-31 06:50:20,2023-10-31 06:50:00,2023-10-31 06:00:00,2023-10-31 00:00:00,2023-10-01 00:00:00
2024-12-02 16:34:21,2024-12-02 16:34:21,2024-12-02 16:34:00,2024-12-02 16:00:00,2024-12-02 00:00:00,2024-12-01 00:00:00


In [111]:
df_times.select(
    'datetime',
    r_1s=pl.col('datetime').dt.round('1s'),
    r_30s=pl.col('datetime').dt.round('30s'),
    r_1h=pl.col('datetime').dt.round('1h'),
    r_1d=pl.col('datetime').dt.round('1d'),
    r_1mo=pl.col('datetime').dt.round('1mo'),
)

datetime,r_1s,r_30s,r_1h,r_1d,r_1mo
datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2022-11-04 01:10:12.200,2022-11-04 01:10:12,2022-11-04 01:10:00,2022-11-04 01:00:00,2022-11-04 00:00:00,2022-11-01 00:00:00
2023-10-31 06:50:20.500,2023-10-31 06:50:21,2023-10-31 06:50:30,2023-10-31 07:00:00,2023-10-31 00:00:00,2023-11-01 00:00:00
2024-12-02 16:34:21,2024-12-02 16:34:21,2024-12-02 16:34:30,2024-12-02 17:00:00,2024-12-03 00:00:00,2024-12-01 00:00:00


In [112]:
df_times.select(
    'datetime',
    o_1s=pl.col('datetime').dt.offset_by('1s'),
    o_30s=pl.col('datetime').dt.offset_by('30s'),
    o_1h=pl.col('datetime').dt.offset_by('1h'),
    o_1d=pl.col('datetime').dt.offset_by('1d'),
    o_1mo=pl.col('datetime').dt.offset_by('1mo'),
)

datetime,o_1s,o_30s,o_1h,o_1d,o_1mo
datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs]
2022-11-04 01:10:12.200,2022-11-04 01:10:13.200,2022-11-04 01:10:42.200,2022-11-04 02:10:12.200,2022-11-05 01:10:12.200,2022-12-04 01:10:12.200
2023-10-31 06:50:20.500,2023-10-31 06:50:21.500,2023-10-31 06:50:50.500,2023-10-31 07:50:20.500,2023-11-01 06:50:20.500,2023-11-30 06:50:20.500
2024-12-02 16:34:21,2024-12-02 16:34:22,2024-12-02 16:34:51,2024-12-02 17:34:21,2024-12-03 16:34:21,2025-01-02 16:34:21


In [113]:
from helper.polars import create_datetime_sample_data
df = create_datetime_sample_data(n=5)
df

start,end,category
datetime[μs],datetime[μs],str
2024-12-10 10:32:14,2024-12-10 10:36:32,"""C"""
2024-12-11 05:21:16,2024-12-11 05:35:20,"""D"""
2024-12-11 14:46:42,2024-12-11 14:48:06,"""A"""
2024-12-11 16:50:09,2024-12-11 17:00:33,"""D"""
2024-12-12 06:24:53,2024-12-12 06:34:56,"""B"""


In [88]:
df.select(pl.col('start') - pl.col('end'))

start
duration[μs]
-11m -20s
-11m -3s
-7m -41s
-13m -18s
-7m -47s


In [7]:
df.select(
    start_ms=pl.col('start').dt.cast_time_unit('ms'),
    start_ns=pl.col('start').dt.cast_time_unit('ns'),
)

start_ms,start_ns
datetime[ms],datetime[ns]
2024-12-10 10:32:14,2024-12-10 10:32:14
2024-12-10 17:29:08,2024-12-10 17:29:08
2024-12-11 11:57:08,2024-12-11 11:57:08
2024-12-12 06:52:42,2024-12-12 06:52:42
2024-12-12 16:24:08,2024-12-12 16:24:08


In [16]:
df_span = df.select(span=pl.col('end') - pl.col('start'))

In [29]:
df_span.select(
    minutes=pl.col('span').dt.total_minutes(),
    seconds=pl.col('span').dt.total_seconds(),
    milliseconds=pl.col('span').dt.total_milliseconds(),
)

minutes,seconds,milliseconds
i64,i64,i64
2,128,128000
14,870,870000
12,769,769000
1,118,118000
7,478,478000


## データ変換

### 文字列との変換

* `read_csv(try_parse_dates=True)`
* `str.to_datetime()`
* `str.to_date()`
* `str.to_time()`
* `str.strptime()`
* `dt.strftime()`

In [9]:
df.write_csv('data/test_time.csv', datetime_format="%Y-%m-%d %H:%M:%S")

with open('data/test_time.csv') as f:
    print(f.read())

start,end,category
2024-12-10 10:32:14,2024-12-10 10:34:22,D
2024-12-10 17:29:08,2024-12-10 17:43:38,C
2024-12-11 11:57:08,2024-12-11 12:09:57,C
2024-12-12 06:52:42,2024-12-12 06:54:40,B
2024-12-12 16:24:08,2024-12-12 16:32:06,C



In [10]:
df = pl.read_csv('data/test_time.csv', try_parse_dates=True)
df

start,end,category
datetime[μs],datetime[μs],str
2024-12-10 10:32:14,2024-12-10 10:34:22,"""D"""
2024-12-10 17:29:08,2024-12-10 17:43:38,"""C"""
2024-12-11 11:57:08,2024-12-11 12:09:57,"""C"""
2024-12-12 06:52:42,2024-12-12 06:54:40,"""B"""
2024-12-12 16:24:08,2024-12-12 16:32:06,"""C"""


https://docs.rs/chrono/latest/chrono/format/strftime/index.html

In [13]:
df_str = df.select(pl.col('start').dt.strftime('%Y%m%d'))
df_datetime = df_str.select(pl.col('start').str.to_datetime('%Y%m%d'))
row(df_str, df_datetime)

start
str
"""20241210"""
"""20241210"""
"""20241211"""
"""20241212"""
"""20241212"""
start
datetime[μs]
2024-12-10 00:00:00
2024-12-10 00:00:00


### Pythonの時間との変換

### epoch

* `pl.from_epoch()`
* `dt.epoch()`

In [58]:
df2 = df.select(
    epoch_s=pl.col('start').dt.epoch('s'),
    epoch_ns=pl.col('start').dt.epoch('ns'),
)
df3 = df2.select(
    start_s = pl.from_epoch('epoch_s', 's'),
    start_ns = pl.from_epoch('epoch_ns', 'ns')
)
row(df.select('start'), df2, df3)

shape: (5, 1) start datetime[μs] 2024-12-10 10:32:14 2024-12-10 12:22:34 2024-12-10 22:36:45 2024-12-11 15:35:33 2024-12-12 07:34:15 
 shape: (5, 2) epoch_s epoch_ns i64 i64 1733826734 1733826734000000000 1733833354 1733833354000000000 1733870205 1733870205000000000 1733931333 1733931333000000000 1733988855 1733988855000000000 
 shape: (5, 2) start_s start_ns datetime[μs] datetime[ns] 2024-12-10 10:32:14 2024-12-10 10:32:14 2024-12-10 12:22:34 2024-12-10 12:22:34 2024-12-10 22:36:45 2024-12-10 22:36:45 2024-12-11 15:35:33 2024-12-11 15:35:33 2024-12-12 07:34:15 2024-12-12 07:34:15